# Data Preprocessing

This notebook contains the preprocessing of the dataset I collected (1946-1963) and the code used to merge Mona Alaert's dataset (1893-1934), Elisabeth Jansen's dataset (1934-1946) and mine. Ultimately creating a dataset combining all the collected data (1893-1963).

## Cleaning up the data

In [1]:
# Loading my dataset (1946-1963)

import pandas as pd

df = pd.read_csv('dataset.csv', sep=';')
df.sample(10)

,Image,Collection,Production,Date,Occasion,Leaflet Language
9127,Scan 2024-03-25 1248-225.jpeg,1959-1960,NaN,NaN,NaN,ned
8495,Scan 2024-03-25 1550-337.jpeg,1959-1960,NaN,NaN,NaN,ned
2012,Scan 2024-02-26 1240-35.jpeg,1950-1951,De Meesterzangers van Nurenberg,17/10/1950,NaN,ned
5387,Scan 2024-03-18 1144-92.jpeg,1955-1956,De Rozenkavalier,23/10/1955,NaN,ned
1163,Scan 2024-02-16 1101-25.jpeg,1948-1949,NaN,NaN,NaN,ned
4146,Scan 2024-03-11 1122-188.jpeg,1953-1954,Madama Butterfly,20/12/1953,NaN,ned
5647,Scan 2024-03-18 1219-108.jpeg,1955-1956,De Heilige van de Bleecker Street,18/02/1956,NaN,ned
2280,Scan 2024-02-26 1334-15.jpeg,1950-1951,NaN,NaN,NaN,ned
10054,Scan 2024-04-29 1008-97.jpeg,1960-1961,NaN,NaN,NaN,ned
11979,Scan 2024-04-29 1742-134.jpeg,1962-1963,NaN,NaN,NaN,ned


In [2]:
# Checking the size of the loaded dataset

print(df.shape)

(12451, 6)


In [3]:
# Removing irrelevant pages, i.e. the production and the date of performance are missing

df = df.dropna(subset=['Production', 'Date'], how='all')
print(f"Dataset now has {len(df)} rows!")

Dataset now has 3366 rows!


In [4]:
# Cells with multiple dates are split so that each row has one date and can be easily converted to datetime
df['Date'] = df['Date'].astype(str)
df['Date'] = df['Date'].str.split(';')
df = df.explode('Date')
df['Date'] = df['Date'].str.strip()

In [5]:
# Converting Date column to datetime

df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

In [6]:
df.sample(10)

,Image,Collection,Production,Date,Occasion,Leaflet Language
11255,Scan 2024-04-29 1242-379.jpeg,1960-1961,Don Carlos,1961-06-11,NaN,ned
5258,Scan 2024-03-11 1549-357.jpeg,1954-1955,Gravin Maritza,1955-06-19,NaN,ned
12315,Scan 2024-04-29 1742-470.jpeg,1962-1963,Tsaar en Timmerman,1963-03-26,galavoorstelling,ned
10899,Scan 2024-04-29 1242-23.jpeg,1960-1961,Parsifal,1961-04-03,NaN,ned
397,Scan 2024-02-14 1232-49.jpeg,1946-1947,Het Lied van Heer Halewijn + Eertijds,1947-06-15,NaN,ned
8119,Scan 2024-03-25 1453-354.jpeg,1959-1960,Tristan en Isolde,1959-10-27,NaN,ned
2723,Scan 2024-02-26 1519-63.jpeg,1951-1952,Tannhauser,1951-12-27,NaN,ned
6031,Scan 2024-03-18 1335-96.jpeg,1956-1957,Chowansjtsjina,1956-10-23,galavoorstelling,ned
12086,Scan 2024-04-29 1742-241.jpeg,1962-1963,Tsaar en Timmerman,1962-12-29,NaN,ned
2234,Scan 2024-02-26 1324-24.jpeg,1950-1951,De Vledermuis,1951-03-13,NaN,ned


In [7]:
# Delete trailing and leading whitespaces

df['Image'] = df['Image'].str.strip() 
df['Collection'] = df['Collection'].str.strip() 
df['Production'] = df['Production'].str.strip() 
df['Occasion'] = df['Occasion'].str.strip()
df['Leaflet Language'] = df['Leaflet Language'].str.strip()

In [8]:
# Create a new column (boolean) to indicate whether the performance was part of a multiple bill (True) or not (False)

multiple_bills = []
for row in df['Production']:
    if '+' in str(row):
        multiple_bills.append(True)
    else:
        multiple_bills.append(False)


df['Multiple Bill'] = multiple_bills
df['Multiple Bill'].value_counts()

False    3154
True      444
Name: Multiple Bill, dtype: int64

In [9]:
# Create a new column assigning a random ID to each performance

import random
df['Performance_ID'] = random.sample(range(100000, 999999), df.shape[0])
df.head()

,Image,Collection,Production,Date,Occasion,Leaflet Language,Multiple Bill,Performance_ID
12,Scan 2024-02-14 1120-13.jpeg,1946-1947,Othello,1946-09-28,openingsvoorstelling,ned,False,988129
20,Scan 2024-02-14 1120-21.jpeg,1946-1947,Othello,1946-09-28,openingsvoorstelling,ned,False,558779
22,Scan 2024-02-14 1120-23.jpeg,1946-1947,Traviata,1946-09-29,NaN,ned,False,187297
24,Scan 2024-02-14 1120-25.jpeg,1946-1947,Faust,1946-09-29,NaN,ned,False,386758
27,Scan 2024-02-14 1120-28.jpeg,1946-1947,De Vogelhandelaar,1946-10-01,galavoorstelling,ned,False,419062


In [10]:
# Cells that include a '+' , i.e. a multiple bill are split so that each row only contains one performance
# Using Performance_ID to keep track of those performances that were part of a multiple bill

df['Production'] = df['Production'].astype(str)
df['Production'] = df['Production'].str.split('+')
df = df.explode('Production')
df['Production'] = df['Production'].str.strip()
df = df.reset_index(drop=True)
df.sample(5)

,Image,Collection,Production,Date,Occasion,Leaflet Language,Multiple Bill,Performance_ID
1035,Scan 2024-02-26 1249-40.jpeg,1950-1951,De Vliegende Hollander,1950-11-26,NaN,ned,False,337717
2662,Scan 2024-03-18 1401-138.jpeg,1956-1957,Vierde Symfonie,1957-03-09,NaN,ned,True,974794
281,Scan 2024-02-14 1408-51.jpeg,1947-1948,Don Giovanni,1947-10-30,NaN,ned,False,367681
3972,Scan 2024-04-29 1742-228.jpeg,1962-1963,De Vledermuis,1962-12-18,galavoorstelling,ned,False,529666
2976,Scan 2024-03-18 1621-137.jpeg,1957-1958,Anna-Marie,1958-06-15,NaN,ned,False,578899


In [11]:
print(f"Dataset now has {len(df)} rows")

Dataset now has 4116 rows


In [12]:
# Spelling normalization: dictionary to keep track of spelling variations

VariatieC = {
    "Bohème" : ["Boheme", "Bohême", "La Boheme"],
    "Het Land van den Glimlach" : ["Het Land van de Glimlach", "Het land van de glimlach"],
    "Norma" : ["La Norma"],
    "De Schim van de Roos" : ["De schim van de Roos"],
    "De Troubadour" : ["Il Trobadore", "Il Trovatore"],
    "Khowanstchina" : ["Chowansjtsjina", "Khovanstsjina"],
    "Taglioni bij Mussette" : ["Taglioni bij Musette"],
    "De Goochelaar van O.L. Vrouw" : ["de goochelaar van O.L. Vrouw"],
    "Pièces Brillantes" : ["Pieces Brillantes"],
    "Prélude à l'après-midi d'un Faune" : ["Prélude à l'après-midi d'un faune", "Prélude à L'Après -midi d'un Faune", "Prélude à L'Après-Midi d'un Faune", "Prélude à L'Après-midi d'un Faune"],
    "Scheherazade" : ["Sheherazade"],
    "De Blauwe Vogel" : ["De blauwe Vogel"],
    "Assepoes" : ["Cinderella"],
    "L'Arlesiana" : ["Arlequinade"],
    "Thaïs" : ["Thais"],
    "De Barbier van Sevilla" : ["Il Barbiere di Seviglia"],
    "Romeo en Julia" : ["Romeo et Juliette", "Romeo en Juliette"],
    "De Graaf van Luxemburg" : ["Graaf van Luxemburg"],
    "Lucia van Lammermoor" : ["Lucie de Lammermoor", "Lucia di Lammermoor"],
    "Cosi fan Tutte" : ["Cosi Fan Tutte"],
    "André Chénier" : ["Andrea Chenier"],
    "La Grande Tentation de Saint-Antoine" : ["De grote verzoeking van Sint Antonius", "De grote Verzoeking van Sint Antonius", "De Grote Verzoeking van Sint Antonius"],
    "Driemeisjeshuis" : ["Drie-Meisjeshuis", "Het Driemeisjeshuis"],
    "Het Gemaskerd Bal" : ["Un Ballo Il Maschera"],
    "Quatuor" : ["Quator"],
    "Tannhauser" : ["Tannhäuser"],
    "Tooverfluit" : ["De Toverfluit", "De Tooverfluit"],
    "Aïda" : ["Aida"],
    "Don Giovanni" : ["Don Juan"],
    "De Parelvisschers" : ["De Parelvissers"],
    "Lakme" : ["Lakmé"],
    "Madame Butterfly" : ["Madama Butterfly"],
    "Salomé" : ["Salome"],
    "Hoffmann's Vertellingen" : ["Hoffmanns Vertellingen", "Hoffmans Vertellingen"],
    "Czaar en Timmerman" : ["Tsaar en Timmerman"],
    "Ballet-suite" : ["Ballet Suite", "Ballet-Suite"],
    "Het Rijngoud" : ["Rijngoud"],
    "Willem Tell" : ["Wilhelm Tell"],
    "De Walkure" : ["De Walküre"],
    "De Doode Oogen" : ["De Dode Ogen"],
    "Boris Godouwnow" : ["Boris Godounow", "Boris Godoenov"],
    "De Vliegende Hollander" : ["De vliegende Hollander"],
    "Namouna" : ["Namouma"],
    "De Wonderbare Mandarijn" : ["De wonderbare Mandarijn"],
    "Tosca" : ["La Tosca"],
    "De Meesterzangers van Nurenberg" : ["De Meesterzangers van Neurenberg"],
    "Thijl Uilenspiegel" : ["Thyl Uilenspiegel"],
    "Traviata" : ["La Traviata"],
    "Czarsdasvorstin" : ["De Czardasvorstin"],
    "De Heilige van Bleecker Street" : ["De Heilige van de Bleecker Street"],
    "In het Witte Paard" : ["In 't Witte Paard"],
    "De Rozenkavalier" : ["De Rozencavalier"],
    "Vierde Symfonie" : ["Vierde Symphonie"],
    "De Regenboog" : ["Regenboog"],
    "Falstaff" : ["Fallstaff"],
    "Het Kasteel van Blauwbaard" : ["Het kasteel van Blauwbaard"],
    "De Antikwaar" : ["L'Antiquaire"],
    "Uitnodiging tot de dans" : ["Uitnodiging tot de Dans"],
    "De Bacchanale" : ["Bacchanale"],
    "Roemeense Rhapsodie" : ["Roemeense Rapsodie"],
    "De Sylphiden" : ["Sylfiden", "De Sylfide", "De Sylfiden"],
    "Jenufa" : ["Jenoefa"],
}

In [13]:
# Function that iterates over the values in the dictionary and returns the key if the title is found in these values
# If the title is not in the dictionary, there is not spelling variation for this title in the dataset

def clean_titles (row):

    for key, values in VariatieC.items():
        if str(row) in str(values):
            #print('it is in the values')
            return key
        elif row == key:
            return key
    else:
        return row 

In [14]:
# Quick test on some frequent variations

a = "Hoffmans Vertellingen"
b = "Tannhäuser"
c = "Salome"

print(clean_titles(a))
print(clean_titles(b))
print(clean_titles(c))

Hoffmann's Vertellingen
Tannhauser
Salomé


In [15]:
# Now applying this function to the entire dataset, the normalized spelling is gathered in a new column called 'Normalized Title'

df['Normalized Title'] = df['Production'].apply(clean_titles)

In [16]:
# Removing duplicates, i.e. the production and date are the same

df = df.drop_duplicates(subset=['Production', 'Date'])
print(f"Dataset now has {len(df)} rows")
df.sample(10)

Dataset now has 4042 rows


,Image,Collection,Production,Date,Occasion,Leaflet Language,Multiple Bill,Performance_ID,Normalized Title
3470,Scan 2024-04-29 1008-15.jpeg,1960-1961,De Vrijschutter,1960-10-02,NaN,ned,False,233802,De Vrijschutter
1670,Scan 2024-02-28 1323-80.jpeg,1952-1953,Traviata,1953-03-08,NaN,ned,False,908829,Traviata
1916,Scan 2024-03-11 1233-29.jpeg,1953-1954,De Parelvissers,1954-03-14,NaN,ned,True,595273,De Parelvisschers
3282,Scan 2024-03-25 1550-405.jpeg,1959-1960,Francesca,1959-12-27,NaN,ned,False,454578,Francesca
2755,Scan 2024-03-18 1432-171.jpeg,1956-1957,Peter Grimes,1957-06-30,NaN,ned,False,575254,Peter Grimes
2430,Scan 2024-03-18 1219-162.jpeg,1955-1956,Tosca,1956-03-15,NaN,ned,True,267229,Tosca
2510,Scan 2024-03-18 1249-155.jpeg,1955-1956,Tannhäuser,1956-06-16,NaN,ned,False,990922,Tannhauser
1561,Scan 2024-02-28 1229-152.jpeg,1952-1953,De Schending van Lucretia,1952-11-22,NaN,ned,False,103718,De Schending van Lucretia
3295,Scan 2024-03-25 1621-95.jpeg,1959-1960,Tosca,1960-01-16,NaN,ned,False,767789,Tosca
295,Scan 2024-02-14 1408-77.jpeg,1947-1948,De Zigeunerbaron,1947-11-16,NaN,ned,False,196305,De Zigeunerbaron


## Adding metadata

Loading a dictionary-formatted txt-file containing additional metadata. This file was created by Mona Allaert and supplemented with new metadata by Elisabeth Jansen and me.
- Key = normalized title
- Value [0] = original language
- Value [1] = composer
- Value [2] = year of first performance
- Value [3] = original title
- Value [4] = genre
- Value [5] = language of normalized title

In [17]:
# Loading the metadata

with open('META3.txt','r', encoding = 'UTF8') as inf:
    Metadata = eval(inf.read())

In [18]:
# Check if the file is formatted correctly

output = []
for key, values in Metadata.items():
    if len(values) != 6:
       output.append(values)
       print(output)

if len(output) == 0:
    print('Looking good!')

Looking good!


In [19]:
# Function that identifies the original language based on the metadata

def add_language (row):
    for key, values in Metadata.items():
        if row == key:
            return values[0]
    else:
        return 'N/A'

# Testing

a = "Madame Butterfly"
b = "Tannhauser"
c = "Quinten Massijs"

print(add_language(a))
print(add_language(b))
print(add_language(c))

ITA
DUI
NL


In [20]:
# Function that identifies the composer based on the metadata

def add_composer (row):
    for key, values in Metadata.items():
        if row == key:
            return values[1]
    else:
        return 'N/A'
    
# Testing

print(add_composer(a))
print(add_composer(b))
print(add_composer(c))

Giacomo Puccini
Richard Wagner
Emile Wambach


In [21]:
# Function that identifies the year of the first performance based on the metadata

def add_premiere (row):
    for key, values in Metadata.items():
        if row == key:
            return values[2]
    else:
        return 'N/A'

print(add_premiere(a))
print(add_premiere(b))
print(add_premiere(c))

1904
1845
1899


In [22]:
# Function that identifies the original title based on the metadata

def add_ortitel (row):
    for key, values in Metadata.items():
        if row == key:
            return values[3]
    else:
        return 'N/A'


print(add_ortitel(a))
print(add_ortitel(b))
print(add_ortitel(c))

Madama Butterfly
Tannhäuser
Quinten Massys


In [23]:
# Function that identifies the genre based on the metadata

def add_genre (row):
    for key, values in Metadata.items():
        if row == key:
            return values[4].lower()
    else:
        return 'N/A'


print(add_genre(a))
print(add_genre(b))
print(add_genre(c))

opera seria
romantische oper
lyrisch drama


In [24]:
# Function that identifies the language of the title on the leaflet based on the metadata

def add_title_lang (row):
    for key, values in Metadata.items():
        if row == key:
            return values[5]
    else:
        return 'N/A'


print(add_title_lang(a))
print(add_title_lang(b))
print(add_title_lang(c))

NL
Unk
Unk


In [25]:
# Add metadata as new columns in the dataframe

df['Original Language'] = df['Normalized Title'].apply(add_language)
df['Composer'] = df['Normalized Title'].apply(add_composer)
df['Original Premiere'] = df['Normalized Title'].apply(add_premiere)
df['Original Title'] = df['Normalized Title'].apply(add_ortitel)
df['Genre'] = df['Normalized Title'].apply(add_genre)
df['Title Language'] = df['Normalized Title'].apply(add_title_lang)

df.sample(10)

,Image,Collection,Production,Date,Occasion,Leaflet Language,Multiple Bill,Performance_ID,Normalized Title,Original Language,Composer,Original Premiere,Original Title,Genre,Title Language
3153,Scan 2024-03-25 1408-29.jpeg,1959-1960,Don Giovanni,1959-09-19,openingsvoorstelling,ned,True,974645,Don Giovanni,ITA,Wolfgang Amadeus Mozart,1787,Don Giovanni,opera buffa,Unk
1444,Scan 2024-02-28 1142-4.jpeg,1951-1952,De Vrijschutter,1952-05-03,NaN,ned,False,813916,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper,NL
1721,Scan 2024-02-28 1354-79.jpeg,1952-1953,De Lustige Boer,1953-05-03,NaN,ned,False,735029,De Lustige Boer,DUI,Leo Fall,1907,Der fidele Bauer,leichte oper,NL
2541,Scan 2024-03-18 1335-79.jpeg,1956-1957,Carmen,1956-10-16,NaN,ned,False,260669,Carmen,FR,Georges Bizet,1875,Carmen,opéra comique,Unk
1283,Scan 2024-02-26 1448-88.jpeg,1951-1952,De Vrijschutter,1951-11-08,NaN,ned,False,247933,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper,NL
666,Scan 2024-02-16 1118-23.jpeg,1948-1949,Parsifal,1949-04-15,NaN,ned,False,831409,Parsifal,DUI,Richard Wagner,1882,Parsifal,music drama,Unk
2044,Scan 2024-03-11 1427-34.jpeg,1954-1955,Kleermaker Wibbel,1954-10-04,NaN,ned,False,567237,Kleermaker Wibbel,DUI,Mark Lothar,1938,Schneider Wibbel,komische opera,NL
2730,Scan 2024-03-18 1432-117.jpeg,1956-1957,Cavalleria Rusticana,1957-06-01,NaN,ned,True,324584,Cavalleria Rusticana,ITA,Pietro Mascagni,1890,Cavalleria rusticana,?,ITA
1582,Scan 2024-02-28 1256-30.jpeg,1952-1953,Madame Butterfly,1952-12-14,NaN,ned,False,611617,Madame Butterfly,ITA,Giacomo Puccini,1904,Madama Butterfly,opera seria,NL
545,Scan 2024-02-16 1040-3.jpeg,1948-1949,Herodias,1948-11-04,NaN,ned,False,954363,Herodias,FR,Jules Massenet,1882,Hérodiade,romantische opera,Unk


## More preprocessing...

In [26]:
# Obtaining the season, an operatic season runs (approximately) from September to May
def get_season(date):
    year = date.year
    if date.month >= 9:
        return f"{year}-{year+1}"
    else:
        return f"{year-1}-{year}"


df['Season'] = df['Date'].apply(get_season)

In [27]:
# Create a column with the directors per season

def get_director(season):
    season_directors = {
        '1946-1947' : 'August Baeyens',
        '1947-1948' : 'August Baeyens',
        '1948-1949' : 'Karel Bogaers',
        '1949-1950' : 'Karel Bogaers',
        '1950-1951' : 'Karel Bogaers',
        '1951-1952' : 'Robert Herberigs',
        '1952-1953' : 'Robert Herberigs',
        '1953-1954' : 'August Baeyens',
        '1954-1955' : 'August Baeyens',
        '1955-1956' : 'August Baeyens',
        '1956-1957' : 'August Baeyens',
        '1957-1958' : 'August Baeyens',
        '1958-1959' : 'Mina Bolotine',
        '1959-1960' : 'Mina Bolotine',
        '1960-1961' : 'Mina Bolotine',
        '1961-1962' : 'Renaat Verbruggen',
        '1962-1963' : 'Renaat Verbruggen',
    }
    return season_directors.get(season, 'unk')  # Default to 'unk' if not found

df['Directors'] = df['Season'].apply(get_director)

df.head()

,Image,Collection,Production,Date,Occasion,Leaflet Language,Multiple Bill,Performance_ID,Normalized Title,Original Language,Composer,Original Premiere,Original Title,Genre,Title Language,Season,Directors
0,Scan 2024-02-14 1120-13.jpeg,1946-1947,Othello,1946-09-28,openingsvoorstelling,ned,False,988129,Othello,TSJ,Antonín Dvořák,1891,Othello,opera seria,Unk,1946-1947,August Baeyens
2,Scan 2024-02-14 1120-23.jpeg,1946-1947,Traviata,1946-09-29,NaN,ned,False,187297,Traviata,ITA,Giuseppe Verdi,1853,La traviata,opera semiseria,Unk,1946-1947,August Baeyens
3,Scan 2024-02-14 1120-25.jpeg,1946-1947,Faust,1946-09-29,NaN,ned,False,386758,Faust,FR,Charles Gounod,1859,Faust,grand opera,Unk,1946-1947,August Baeyens
4,Scan 2024-02-14 1120-28.jpeg,1946-1947,De Vogelhandelaar,1946-10-01,galavoorstelling,ned,False,419062,De Vogelhandelaar,DUI,Carl Zeller,1891,Der Vogelhändler,operette,NL,1946-1947,August Baeyens
5,Scan 2024-02-14 1120-30.jpeg,1946-1947,Othello,1946-10-03,NaN,ned,False,356547,Othello,TSJ,Antonín Dvořák,1891,Othello,opera seria,Unk,1946-1947,August Baeyens


In [28]:
# Saving the cleaned dataset

# df2.to_csv('cleaned_dataset.csv', encoding = 'utf-8')

## Merging datasets

In [29]:
# Loading the cleaned dataset containing Mona's and Elisabeth's data

df_1893_1946 = pd.read_csv("Merged_Dataset.csv", encoding='utf-8', index_col=False)
df_1893_1946.head()

,Unnamed: 0,Image,Collection,Production,Date,Occasion,Multiple bill,Performances_ID,Normalised title,Original language,Composer,Original premiere,Original title,genre,Season,Directors,Title_Language,Leaflet_Language,Opmerking
0,0,K.V.O. programmas 1893-97_00005-20211201_11335...,1893-1897,De Vrijschutter,1893-10-03,Eerste vertoning,False,903641,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper,1893-1894,Edward Keurvels & Henry Fontaine,NL,NaN,NaN
1,1,K.V.O. programmas 1893-97_00009-20211201_11343...,1893-1897,Willem Tell,1893-10-05,Eerste vertoning,False,316913,Willem Tell,DUI,Carl Reinecke,1971,Musik zu Schiller’s 'Wilhelm Tell,romantische oper,1893-1894,Edward Keurvels & Henry Fontaine,Unk,NaN,NaN
2,2,K.V.O. programmas 1893-97_00011-20211201_11345...,1893-1897,De Vrijschutter,1893-10-10,NaN,False,135474,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper,1893-1894,Edward Keurvels & Henry Fontaine,NL,NaN,NaN
3,3,K.V.O. programmas 1893-97_00013-20211201_11351...,1893-1897,Willem Tell,1893-10-12,NaN,False,520507,Willem Tell,DUI,Carl Reinecke,1971,Musik zu Schiller’s 'Wilhelm Tell,romantische oper,1893-1894,Edward Keurvels & Henry Fontaine,Unk,NaN,NaN
4,4,K.V.O. programmas 1893-97_00015-20211201_11352...,1893-1897,Charlotte Corday,1893-11-16,NaN,False,446314,Charlotte Corday,NL,Peter Benoit,?,Charlotte Corday,lyrisch drama,1893-1894,Edward Keurvels & Henry Fontaine,Unk,NaN,NaN


In [30]:
# Checking shape of their merged dataset

df_1893_1946.shape

(6767, 19)

In [31]:
# Making sure the columns have the same name (column 'opmerking' is removed because it is not relevant for my thesis)

df_1893_1946 = df_1893_1946.rename(columns={"Multiple bill" : "Multiple Bill", 
                                            "Performances_ID" : "Performance_ID", 
                                            "Normalised title" : "Normalized Title",
                                            "Original language" : "Original Language",
                                            "Original premiere" : "Original Premiere",
                                            "Original title" : "Original Title",
                                            "genre" : "Genre",
                                            "Title_Language" : "Title Language",
                                            "Leaflet_Language" : "Leaflet Language"
                                           })

df_1893_1946 = df_1893_1946.drop(columns=["Opmerking"])

df_1893_1946.head()

,Unnamed: 0,Image,Collection,Production,Date,Occasion,Multiple Bill,Performance_ID,Normalized Title,Original Language,Composer,Original Premiere,Original Title,Genre,Season,Directors,Title Language,Leaflet Language
0,0,K.V.O. programmas 1893-97_00005-20211201_11335...,1893-1897,De Vrijschutter,1893-10-03,Eerste vertoning,False,903641,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper,1893-1894,Edward Keurvels & Henry Fontaine,NL,NaN
1,1,K.V.O. programmas 1893-97_00009-20211201_11343...,1893-1897,Willem Tell,1893-10-05,Eerste vertoning,False,316913,Willem Tell,DUI,Carl Reinecke,1971,Musik zu Schiller’s 'Wilhelm Tell,romantische oper,1893-1894,Edward Keurvels & Henry Fontaine,Unk,NaN
2,2,K.V.O. programmas 1893-97_00011-20211201_11345...,1893-1897,De Vrijschutter,1893-10-10,NaN,False,135474,De Vrijschutter,DUI,Carl Maria von Weber,1821,Der Freischütz,romantische oper,1893-1894,Edward Keurvels & Henry Fontaine,NL,NaN
3,3,K.V.O. programmas 1893-97_00013-20211201_11351...,1893-1897,Willem Tell,1893-10-12,NaN,False,520507,Willem Tell,DUI,Carl Reinecke,1971,Musik zu Schiller’s 'Wilhelm Tell,romantische oper,1893-1894,Edward Keurvels & Henry Fontaine,Unk,NaN
4,4,K.V.O. programmas 1893-97_00015-20211201_11352...,1893-1897,Charlotte Corday,1893-11-16,NaN,False,446314,Charlotte Corday,NL,Peter Benoit,?,Charlotte Corday,lyrisch drama,1893-1894,Edward Keurvels & Henry Fontaine,Unk,NaN


In [32]:
# Checking shape of my dataset

df.shape

(4042, 17)

In [33]:
# Combining the dataset

df = pd.concat([df_1893_1946, df], ignore_index=True)
df.sample(10)

,Unnamed: 0,Image,Collection,Production,Date,Occasion,Multiple Bill,Performance_ID,Normalized Title,Original Language,Composer,Original Premiere,Original Title,Genre,Season,Directors,Title Language,Leaflet Language
8983,NaN,Scan 2024-03-11 1549-294.jpeg,1954-1955,Zwarte Viooltjes,1955-05-17 00:00:00,NaN,False,507563,Zwarte Viooltjes,NL,August Baeyens,1937,Zwarte Viooltjes,operette,1954-1955,August Baeyens,NL,ned
204,204.0,K.V.O. programmas 1893-97_00440-20211201_13121...,1893-1897,Czaar en Timmerman,1896-11-26,NaN,False,122524,Czaar en Timmerman,DUI,Albert Lortzing,1837,Zar und Zimmermann,zangspel,1896-1897,Edward Keurvels & Henry Fontaine,NL,NaN
3698,3698.0,1929-1930 bis_00311-20220209_140906.jpg,1929-1930,Siegfried,1930-04-06,NaN,False,154250,Siegfried,DUI,Richard Wagner,1871,Siegfried,gesamtkunstwerk,1929-1930,Fé Derickx & Bernard Tokkie,Unk,NaN
6728,6728.0,0430_1945-1946_00430-20230310_133348.jpg,1945/1946,Aida,1946-06-09,NaN,False,154690,Aïda,ITA,Giuseppe Verdi,1872,Aïda,opera,1945-1946,August L. Baeyens,Unk,Dutch
396,396.0,1897-1900 Nederlandsch Lyrisch Tooneel. Progra...,1897-1900,Cleopatra,1899-12-03,NaN,False,114344,Cleopatra,DEE,August Enna,1894,Cleopatra,opera seria,1899-1900,Karel van Walle,Unk,NaN
10395,NaN,Scan 2024-04-29 1403-103.jpeg,1961-1962,Don Pasquale,1961-10-01 00:00:00,NaN,True,309953,Don Pasquale,ITA,Gaetano Donizetti,1843,Don Pasquale,komische opera,1961-1962,Renaat Verbruggen,unk,ned
10019,NaN,Scan 2024-03-25 1204-36.jpeg,1959-1960,Boheme,1960-02-07 00:00:00,NaN,False,760074,Bohème,ITA,Giacomo Puccini,1897,La bohème,verismo opera,1959-1960,Mina Bolotine,Unk,ned
4679,4679.0,1935-1936_00178-20230218_141118.jpg,1935-1936,Boheme,1936-01-09,NaN,False,197220,Bohème,ITA,Giacomo Puccini,1897,La bohème,verismo opera,1935-1936,Jef Sterkens,Unk,Dutch
1990,1990.0,1919-1920_00043-20220207_102228.jpg,1919-1920,Figaro's Bruiloft,1919-11-27,NaN,False,278749,Figaro's Bruiloft,ITA,Wolfgang Amadeus Mozart,1786,Le nozze di Figaro,oepra buffa,1919-1920,Henry Fontaine,NL,NaN
7890,NaN,Scan 2024-02-26 1324-40.jpeg,1950-1951,Houten Prins,1951-03-24 00:00:00,NaN,True,326994,Houten Prins,HON,Béla Bartók,1917,A fából faragott királyfi,ballet,1950-1951,Karel Bogaers,NL,ned


In [34]:
# Again checking the shape

df.shape

(10809, 18)

In [35]:
# Delete all trailing and leading whitespace

for column in df.columns:
    try:
        df[column] = df[column].str.strip()
    except AttributeError:
        continue

In [36]:
# Convert the date to datetime

df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

In [37]:
# Save merged dataframe (1893-1963)

# df.to_csv('complete_dataset.csv', encoding = 'utf-8')